<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic

In [ ]:
import os
from bertopic import BERTopic
import json

model = BERTopic(language="english")
#model.visualize_topics()

RESULTS_HOME = '/kaggle/input/instance-json/results/results'
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [ ]:
'''topic, prob = topic_model.transform("This is an incredible movie!")
topic_model.topic_labels_[topic[0]]'''
from tqdm import tqdm

In [ ]:
instances_file_path = '/kaggle/input/instance-json/instances.jsonl'

instances = []

with open(instances_file_path, 'r') as file:
    for line in file:
        instance_data = json.loads(line)
        instances.append(instance_data)

#filtered_instances = [inst for inst in instances if inst["instance"] != "mastodon.social"]
filtered_instances = [inst for inst in instances]

valid_instances = []

for instance in filtered_instances:
    instance_file_path = os.path.join(RESULTS_HOME, f"{instance['instance']}.json")
    if os.path.isfile(instance_file_path):
        valid_instances.append(f"{instance['instance']}.json")
    if len(valid_instances) == 15:
        break

print(valid_instances)


In [ ]:
def process_batch(posts_batch):
    t, _ = topic_model.transform(posts_batch)
    return [topic_model.topic_labels_[i] for i in t]

def process_batch2(posts_batch):
    t, probs = model.fit_transform(posts_batch)
    #return [topic_model.topic_labels_[i] for i in t]

In [ ]:
def process_topics():    
    for instance_name in tqdm(valid_instances, desc="Processing Instance"):
        aligned_count = 0
        total_count = 0
        instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
        posts = [post['text'] for post in tqdm(instance['records'], desc='Pull post' ) if post['language'] == 'en']
        aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
        tags = []
        for tagl in tqdm(aux, desc='Processing tags'):
            for tag in tagl:
                tags.append(tag['name'])

        #print(f"{instance_name}",tags)

        batch_size = 10000
        batches = [posts[i:i + batch_size] for i in range(0, len(posts), batch_size)]

        topic_path = os.path.join('/kaggle/working', f'fullgas_topics_{instance_name}.txt')

        all_topics = []

        if os.path.exists(topic_path):
            # Se il file esiste, carica i topic dal file
            with open(topic_path, 'r') as f:
                all_topics = [line.strip() for line in f.readlines()]
            print(f'Topics loaded from {topic_path}')
        else:
            # Se il file non esiste, genera i topic e salvali nel file
            for batch in tqdm(batches, desc='Processing batches'):
                topics = process_batch(batch)
                all_topics.extend(topics)

                with open(topic_path, 'a') as f:
                    for topic in topics:
                        f.write(f'{topic}\n')

        if len(tags) > 0:
            total_count += len(tags)
            for tag in tqdm(tags, desc='Checking tag alignment'):
                for topic in all_topics:
                    if tag in topic:
                        aligned_count += 1
                        break
        else:
            total_count = 1
        output_path = os.path.join('/kaggle/working', f'count_results{instance_name}.txt')
        with open(output_path, 'w') as file:
            file.write(f'Aligned Count: {aligned_count}\n')
            file.write(f'Total Count: {total_count}\n')
            file.write(f'Alignment / Total : {aligned_count / total_count:.4f}\n')

        print(aligned_count)
        print(total_count)
        print(aligned_count/total_count)

In [ ]:
def fit_topic():    
    for instance_name in tqdm(valid_instances, desc="Processing Instance"):
        aligned_count = 0
        total_count = 0
        instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
        posts = [post['text'] for post in tqdm(instance['records'], desc='Pull post' ) if post['language'] == 'en']
        aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
        tags = []
        for tagl in tqdm(aux, desc='Processing tags'):
            for tag in tagl:
                tags.append(tag['name'])

        #print(f"{instance_name}",tags)

        batch_size = 10000
        batches = [posts[i:i + batch_size] for i in range(0, len(posts), batch_size)]

        topic_path = os.path.join('/kaggle/working', f'fullgas_topics_{instance_name}.txt')

        all_topics = []

        if os.path.exists(topic_path):
            # Se il file esiste, carica i topic dal file
            with open(topic_path, 'r') as f:
                all_topics = [line.strip() for line in f.readlines()]
            print(f'Topics loaded from {topic_path}')
        else:
            # Se il file non esiste, genera i topic e salvali nel file
            for batch in tqdm(batches, desc='Processing batches'):
                topics = process_batch(batch)
                all_topics.extend(topics)

                with open(topic_path, 'a') as f:
                    for topic in topics:
                        f.write(f'{topic}\n')

        if len(tags) > 0:
            total_count += len(tags)
            for tag in tqdm(tags, desc='Checking tag alignment'):
                for topic in all_topics:
                    if tag in topic:
                        aligned_count += 1
                        break
        else:
            total_count = 1
        output_path = os.path.join('/kaggle/working', f'count_results{instance_name}.txt')
        with open(output_path, 'w') as file:
            file.write(f'Aligned Count: {aligned_count}\n')
            file.write(f'Total Count: {total_count}\n')
            file.write(f'Alignment / Total : {aligned_count / total_count:.4f}\n')

        print(aligned_count)
        print(total_count)
        print(aligned_count/total_count)

In [ ]:
def tot_post():
    c = 0
    for instance_name in tqdm(os.listdir(RESULTS_HOME)):
        instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
        posts = [post['text'] for post in instance['records'] if post['language'] == 'en']
        aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
        tags = []
        for tagl in aux:
            for tag in tagl:
                tags.append(tag['name'])
        if len(tags)>0:
            c+= len(posts)

    print(c)
#Numero di post totali da analizzare 18387780

In [ ]:
#!rm -r /kaggle/working
#!zip -r scores.zip /kaggle/working